In [2]:
#Sample inputs
user_age = 25
menstrual_phase = "luteal"
workout_goal = "lose fat and tone muscle"
diet = "vegetarian"

In [3]:
from pymilvus import model


embedding_fn = model.DefaultEmbeddingFunction()

docs = [
    "Follicular Phase, is the time you start feeling the buzz as your energy returns.",
"Estrogen levels are the most high during follicular phase",
"During follicular phase you are more likely to handle increasingly challenging workouts",
"Rowing, Running, Walking (incline walks & power walks), Jumping rope, Cardio-focused, classes, Cycling/biking/spinning, Resistance Training are good exercises during your follicular phase",
"Bootcamps (burpees), HIIT Circuits & tabata workouts, Indoor Cycling Classes, Track/Sprint, Workouts & Intervals, Runs, Kickboxing & Boxing, Power Yoga, Resistance Training, and Kettlebell workouts, are good exercises during your ovulation phase",
"Your body is programmed to get you moving and grooving right during ovulation phase",
"You are more social & you are more jazzy about everything, including exercise during ovulation.",
"During ovulation as estrogen and testosterone hit their peak levels, you have got energy to churn",
"During your first half of luteal phase, You might still have lots of energy coming off ovulation, so it is ok to keep your workouts challenging and maximize lean muscle gains by focusing on slow strength training.",
"During first half of luteal phase out HIIT and focus on intention of your movements,not intensity.",
"During second half of luteal do Pilates, Barre, Yoga, Walks, More rest days, and Stretching, and other low intensity workouts.",
"As your period approaches you might be very tired. Do not be afraid to take a short nap in place of your allotted workout time.",
"During your menstrual phase, do Walking, Slow yoga (it can help with cramping), Pilates, More rest days, Foam rolling, Stretching, Mobility, Naps (quick 20-minute snooze).",
"Take it easy during your period, HIIT will backfire by turning on fat-storage and encourage muscle wasting.",
"During your period wake up slower with a cup of tea and enjoy some gentle movement afterwards or wait until lunch or after work to get some exercise in.",
"During your period it is time to focus on restorative movements, and low and slow activity.",
"In your 20s you should build Strength and Endurance and your goals would be to Build muscle, improve endurance, and maintain flexibility.",
"During your 30s you should focus on Maintaining Strength and Preventing Stress and you Goals would be to Maintain muscle mass, reduce stress, and increase endurance.",
"In your 40s, focus on hormonal Balance and Bone Health and your Goals should be to Improve bone density, preserve muscle mass, and manage hormonal changes.",
"In your 50s, focus on Staying Active and Reducing Risk of Chronic Conditions and your Goals should be to Increase cardiovascular health, improve flexibility, and reduce the risk of chronic illnesses.",
"During menstruation, eat Iron-Rich Foods such as Spinach, kale, red meat, lentils, and fortified cereals, more Vitamin C to absorb iron such as oranges, strawberries, or bell peppers, Anti-Inflammatory Foods like Fatty fish (salmon, mackerel), walnuts, flaxseeds, and chia seeds., and Hydration such as Herbal teas.",
"During follicular phase, eat protein rich foods like Lean meats, eggs, tofu, or legumes for cell repair, more B Vitamins such as Whole grains, eggs, and nuts to support energy production, Omega-3s like Fatty fish or walnuts to reduce inflammation, and Leafy Greens For folate.",
"During ovulatory phase, eat antioxidants such as Berries, citrus fruits, and dark chocolate to combat oxidative stress, have Zinc through Pumpkin seeds, shellfish, or chickpeas to support reproductive health, and Fiber from Cruciferous vegetables (broccoli, cauliflower) to aid hormone metabolism.",
"During luteal phase, have more magnesium from Dark chocolate, bananas, spinach, and almonds to ease cramps and improve mood, more Complex Carbs from Brown rice, oats, and quinoa to stabilize blood sugar and reduce cravings, increase Calcium & Vitamin D through Yogurt, fortified plant milk, or leafy greens for bone health and mood regulation, and have Herbal Teas such as Chamomile or turmeric tea to reduce bloating and promote relaxation.",
"Menstrual, luteal, follicular, and ovulation phase all are a week long"
]

vectors = embedding_fn.encode_documents(docs)
print("Dim:", embedding_fn.dim, vectors[0].shape)

data = [
    {"id": i, "vector": vectors[i], "text": docs[i], "subject": "menstruation"}
    for i in range(len(vectors))
]

print("Data has", len(data), "entities, each with fields: ", data[0].keys())
print("Vector dim:", len(data[0]["vector"]))

Dim: 768 (768,)
Data has 25 entities, each with fields:  dict_keys(['id', 'vector', 'text', 'subject'])
Vector dim: 768


In [4]:
from pymilvus import MilvusClient

client = MilvusClient("milvus_demo.db")
if client.has_collection(collection_name="demo_collection"):
    client.drop_collection(collection_name="demo_collection")
client.create_collection(
    collection_name="demo_collection",
    dimension=768,
)
res = client.insert(collection_name="demo_collection", data=data)

print(res)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{'insert_count': 25, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]}


In [5]:
def retrieve_top_articles(query, top_k=5):
    client = MilvusClient("milvus_demo.db")
    query_embedding=embedding_fn.encode_queries([query])

    res = client.search(
        collection_name="demo_collection",
        data=query_embedding,
        limit=top_k,
        output_fields=["text", "subject"],
    )
    return res

In [12]:
import os
from mistralai import Mistral
from dotenv import load_dotenv

key = os.getenv('MISTRAL_API_KEY')

# api_key = os.environ["MISTRAL_API_KEY"]

def generate_text_based_on_articles(top_articles, query):
    model = "mistral-large-latest"
    client = Mistral(api_key="4S2tKeF1reZMQrZdwTcZ2Zq9xG6r63C7")
    if top_articles:
        input_text = f"""
        Answer the following question based on the provided articles:

        Question: {query}

        Articles:
        {top_articles}

        Please provide a precise and factual answer based on these articles. Avoid adding information not present in the articles.
        """
    else:
        input_text = query

    response = client.chat.complete(
        model=model,
        messages=[
            {
                "role": "system",
                "content": "You are an expert assistant trained to provide factual and precise answers based on provided context."
            },
            {
                "role": "user",
                "content": input_text,
            },
        ]
    )

    generated_text = response.choices[0].message.content
    return generated_text


In [13]:
query = f"Give me a step by step workout plan based on my age: {user_age} and workout goal: {workout_goal}. Please be very explicit about catering a workout plan based on my menstrual phase: {menstrual_phase}"

top_articles = retrieve_top_articles(query)
generated_text = generate_text_based_on_articles(top_articles, query)

print(f"Generated Text:\n{generated_text}")

Generated Text:
Based on the provided articles, here's a step-by-step workout plan for a 25-year-old aiming to lose fat and tone muscle, tailored to the luteal phase of the menstrual cycle:

### Luteal Phase Workout Plan

#### Week 1-2 (Early Luteal Phase)
During the first half of the luteal phase, you might still have lots of energy coming off ovulation. Therefore, it is okay to keep your workouts challenging and maximize lean muscle gains by focusing on slow strength training.

**Monday:**
- **Slow Strength Training:** Focus on compound movements like squats, deadlifts, and bench presses. Perform each exercise slowly and with control to maximize muscle engagement.

**Tuesday:**
- **Resistance Training:** Incorporate exercises like lunges, push-ups, and pull-ups. Maintain a slow and controlled pace.

**Wednesday:**
- **Power Yoga:** This will help maintain flexibility and strength while keeping the intensity high.

**Thursday:**
- **Kettlebell Workouts:** Use kettlebells for exercises